In [2]:
import os
import pickle
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
from sklearn.metrics import accuracy_score

In [4]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

In [5]:
data_dict = 'data_full_0411.pickle'
with open(data_dict, 'rb') as handle:
    data= pickle.load(handle)

for data_dt in data:
    Seq_ind, Seq_ind_label = data[data_dt][4]
    for fol in range(4):
        Seq_val, Seq_val_label = data[data_dt][fol]
        Seq_val_label = np.float32(Seq_val_label)

In [ ]:
from models import *

def load_model(args_model):
    if args_model == 0:
        model = org_model()
        model_name = 'miraw_org'
    elif args_model == 1:
        model = model_v1()
        model_name = 'model_v1'
    elif args_model == 2:
        model = model_v2()
        model_name = 'model_v2'
    elif args_model == 3:
        model = model_v3()
        model_name = 'model_v3'
    elif args_model == 4:
        model = model_v4()
        model_name = 'model_v4'
    elif args_model == 5:
        model = model_v5()
        model_name = 'model_v5'    
    else:
        assert False

    model.compile(
        loss='binary_crossentropy', 
        optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3), 
        metrics=[
            'accuracy',
            tf.keras.metrics.F1Score(threshold=0.5, average='macro')]
    )    
    return model, model_name

c:\Users\a3029\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 66)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 66, 5)     │         25 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 66, 512)   │     28,672 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 66, 512)   │      2,048 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 66, 512)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 66, 512)   │          0 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ dropout[0][0]     │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     65,664 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 128)       │          0 │ dense[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │     66,048 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 512)       │          0 │ dense_1[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 512)    │          0 │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 66, 512)   │          0 │ dropout[0][0],    │
│                     │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 13, 512)   │          0 │ multiply[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 9, 512)    │          0 │ multiply[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 5, 512)    │          0 │ multiply[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 27, 512)   │          0 │ max_pooling1d[0]… │
│ (Concatenate)       │                   │            │ max_pooling1d_1[… │
│                     │                   │            │ max_pooling1d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 27, 512)   │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 331,562 (1.26 MB)

 Trainable params: 330,282 (1.26 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [7]:
def load_fol_list_w(data_dt, model_name, fol):
    save_model_path = os.path.join('Trained_model', data_dt, model_name + '_FOL' + str(fol) )
    list_w = []
    for root, dirs, files in os.walk(save_model_path):
        for file in files:
            if file.endswith('.h5'):
                model_weights = os.path.join(root, file)
                json_f = model_weights.replace('_best_acc.h5', '.json')
                assert os.path.exists(model_weights)
                assert os.path.exists(json_f)
                list_w.append(model_weights)
    return list_w

In [8]:
data_res = {}
for args_model in range(7):
    model, model_name = load_model(args_model)
    data_res[model_name] = {}
    for data_dt in data:
#    for data_dt in ['DeepMirTar']:
    #    data_dt = 'miRaw'
        data_res[model_name][data_dt] = {}
        
        Seq_ind, Seq_ind_label = data[data_dt][4]
        Seq_ind_label = np.float32(Seq_ind_label)            

        Seq_ind_2, Seq_ind_label_2 = data[data_dt][5]
        Seq_ind_label_2 = np.float32(Seq_ind_label_2)            

        for fol in range(4):

            print(model_name, data_dt, fol)
            
            data_res[model_name][data_dt][fol] = {}
            data_res[model_name][data_dt][fol]['val'] = []
            data_res[model_name][data_dt][fol]['ind_1'] = []
            data_res[model_name][data_dt][fol]['ind_2'] = []

            Seq_val, Seq_val_label = data[data_dt][fol]
            Seq_val_label = np.float32(Seq_val_label)            
            list_w = load_fol_list_w(data_dt, model_name, fol)
            for weights in list_w:
                model.load_weights(weights)
                
                res = model.predict(Seq_val, verbose=0, batch_size=1024)
                res = np.squeeze(res)
                data_res[model_name][data_dt][fol]['val'].append(res)

                res_ind = model.predict(Seq_ind, verbose=0, batch_size=1024)
                res_ind = np.squeeze(res_ind)
                data_res[model_name][data_dt][fol]['ind_1'].append(res_ind)

                res_ind_2 = model.predict(Seq_ind_2, verbose=0, batch_size=1024)
                res_ind_2 = np.squeeze(res_ind_2)
                data_res[model_name][data_dt][fol]['ind_2'].append(res_ind_2)

with open('res_full_0415.pickle', 'wb') as handle:
    pickle.dump(data_res, handle, protocol=pickle.HIGHEST_PROTOCOL)


miraw_org DeepMirTar 0
miraw_org DeepMirTar 1
miraw_org DeepMirTar 2
miraw_org DeepMirTar 3
miraw_org miRaw 0
miraw_org miRaw 1
miraw_org miRaw 2
miraw_org miRaw 3
model_v1 DeepMirTar 0
model_v1 DeepMirTar 1
model_v1 DeepMirTar 2
model_v1 DeepMirTar 3
model_v1 miRaw 0
model_v1 miRaw 1
model_v1 miRaw 2
model_v1 miRaw 3
model_v2 DeepMirTar 0
model_v2 DeepMirTar 1
model_v2 DeepMirTar 2
model_v2 DeepMirTar 3
model_v2 miRaw 0
model_v2 miRaw 1
model_v2 miRaw 2
model_v2 miRaw 3
model_v3 DeepMirTar 0
model_v3 DeepMirTar 1
model_v3 DeepMirTar 2
model_v3 DeepMirTar 3
model_v3 miRaw 0
model_v3 miRaw 1
model_v3 miRaw 2
model_v3 miRaw 3
model_v4 DeepMirTar 0
model_v4 DeepMirTar 1
model_v4 DeepMirTar 2
model_v4 DeepMirTar 3
model_v4 miRaw 0
model_v4 miRaw 1
model_v4 miRaw 2
model_v4 miRaw 3
model_v5 DeepMirTar 0
model_v5 DeepMirTar 1
model_v5 DeepMirTar 2
model_v5 DeepMirTar 3
model_v5 miRaw 0


KeyboardInterrupt: 

In [ ]:
data_dt, model_name, fol

('miRaw', 'model_v5', 3)

In [ ]:
data_report = {}
data_report['model_name'] = []
data_report['data_dt'] = []
data_report['fol'] = []
data_report['val'] = []
data_report['ind_1'] = []
data_report['ind_2'] = []

for model_name in data_res:
    
    for data_dt in data:
#    for data_dt in ['DeepMirTar']:
        
        Seq_ind, Seq_ind_label = data[data_dt][4]
        Seq_ind_label = np.squeeze(np.int64(Seq_ind_label))            
        Seq_ind_2, Seq_ind_label_2 = data[data_dt][5]
        Seq_ind_label_2 = np.squeeze(np.int64(Seq_ind_label_2))          
        for fol in range(4):
            Seq_val, Seq_val_label = data[data_dt][fol]
            Seq_val_label = np.int64(Seq_val_label)          
            val = data_res[model_name][data_dt][fol]['val']
            val = np.array(val)
            pval = np.int64(val>=0.5)
            acc_val = [accuracy_score(Seq_val_label, p) for p in pval]
            
            acc_val_avg = np.round(np.average(acc_val)*100,3)
            acc_val_std = np.round(np.std(acc_val)*100,3)
            val_res_str = str(acc_val_avg) + ' ± ' + str(acc_val_std)
            print(model_name, data_dt, fol, 'val', val_res_str)
    
            ind_1 = data_res[model_name][data_dt][fol]['ind_1']
            ind_1 = np.array(ind_1)
            pind_1 = np.int64(ind_1>=0.5)
            acc_ind_1 = [accuracy_score(Seq_ind_label, p) for p in pind_1]

            acc_ind_1_avg = np.round(np.average(acc_ind_1)*100,3)
            acc_ind_1_std = np.round(np.std(acc_ind_1)*100,3)
            ind_1_res_str = str(acc_ind_1_avg) + ' ± ' + str(acc_ind_1_std)
            print(model_name, data_dt, fol, 'ind_1', ind_1_res_str)
    
            ind_2 = data_res[model_name][data_dt][fol]['ind_2']
            ind_2 = np.array(ind_2)
            pind_2 = np.int64(ind_2>0.5)
            acc_ind_2 = [accuracy_score(Seq_ind_label_2, p) for p in pind_2]
            
            acc_ind_2_avg = np.round(np.average(acc_ind_2)*100,3)
            acc_ind_2_std = np.round(np.std(acc_ind_2)*100,3)
            ind_2_res_str = str(acc_ind_2_avg) + ' ± ' + str(acc_ind_2_std)
            print(model_name, data_dt, fol, 'ind_2', ind_2_res_str)

            data_report['model_name'].append(model_name)
            data_report['data_dt'].append(data_dt)
            data_report['fol'].append(fol)
            data_report['val'].append(val_res_str)
            data_report['ind_1'].append(ind_1_res_str)
            data_report['ind_2'].append(ind_2_res_str)

datatoexcel = pd.ExcelWriter('Result_0415.xlsx')
pd.DataFrame(data_report).to_excel(datatoexcel)
# save the excel
datatoexcel.close()
print('DataFrame is written to Excel File successfully.')

NameError: name 'data_res' is not defined

In [ ]:
data_dt

'miRaw'

In [ ]:
Seq_ind, Seq_ind_label = data['DeepMirTar'][3]
Seq_ind_label = np.int64(Seq_ind_label)            
Seq_ind_label = np.squeeze(Seq_ind_label)
print(Seq_ind_label.shape)

Seq_ind_2, Seq_ind_label_2 = data['DeepMirTar'][5]
Seq_ind_label_2 = np.int64(Seq_ind_label_2)  
Seq_ind_label_2 = np.squeeze(Seq_ind_label_2)
print(Seq_ind_label_2.shape)

ind_1 = model.predict(Seq_ind, batch_size=1024)
ind_1 = np.squeeze(ind_1)
ind_1 = np.array(ind_1)
pind_1 = np.int64(ind_1>=0.5)
print(pind_1.shape)

acc_ind_1 = accuracy_score(Seq_ind_label, pind_1)

acc_ind_1_avg = np.round(np.average(acc_ind_1)*100,3)
acc_ind_1_std = np.round(np.std(acc_ind_1)*100,3)
ind_1_res_str = str(acc_ind_1_avg) + ' ± ' + str(acc_ind_1_std)
print(model_name, data_dt, fol, 'ind_1', ind_1_res_str)

(1541,)
(48,)
2/2 [==============================] - 0s 7ms/step
(1541,)
model_v5 miRaw 3 ind_1 51.46 ± 0.0
